In [ ]:
import builtins
import line_profiler as lp
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
def execfile(filename, globals=None, locals=None):
    exec_ = getattr(builtins, "exec")
    with open(filename, "rb") as f:
        exec_(compile(f.read(), filename, "exec"), globals, locals)


def run_experiment(params, repeat=3):
    prof = lp.LineProfiler()
    builtins.__dict__["profile"] = prof

    results = {"rust": {}, "python": {}, "generate": {}}
    for (N, k) in params:
        rr, pp = 0, 0
        for _ in range(repeat):
            ns = {"N": N, "k": k}
            execfile("benchmark.py", ns, ns)
            s = prof.get_stats()
            t = s.timings

            rust = [k for k in t.keys() if k[2] == "rust"][0]
            python = [k for k in t.keys() if k[2] == "python"][0]

            rr += t[rust][0][2] * s.unit
            pp += t[python][0][2] * s.unit

        results["rust"][(N, k)] = rr / repeat
        results["python"][(N, k)] = pp / repeat

        yield results

def fixed_k(k, a, b):
    """
    Generate params with a fixed value of k
    
    V = N, E = N*k/2
    """
    return [(int(2 ** (i / 2)), k) for i in range(a, b)]

def proportional_k(ratio, a, b):
    """ Generate params with a proportional value of k
    
    V = N, k = ratio*N, E = N^2*ratio/2
    """
    return [(int(2 ** (i / 2)), int(int(2 ** (i / 2)) * ratio)) for i in range(a, b)]

def fixed_n(N, a, b):
    """
    Generate params with a fixed value of N
    
    V = N, E = N*k/2
    """
    return [(N, int(2 ** (i / 2))) for i in range(a, b)]
    
    
def plot_data(data, fig, x_key='N'):
    df = pd.DataFrame.from_dict(data)
    if x_key=='N':
        x = [xx[0] for xx in np.array(df.index)]
    else:
        x = [xx[1] for xx in np.array(df.index)]
    fig.data[0].x = x
    fig.data[0].y = np.array(df.rust)

    fig.data[1].x = x
    fig.data[1].y = np.array(df.python)

    fig.data[2].x = x
    fig.data[2].y = np.array(df.rust/df.python)

    
def setup_plots(title, x_label):
    fig = go.FigureWidget(make_subplots(rows=2, cols=1, subplot_titles =["Performance", "Rust relative time (Cython = 1.0)"]))

    
    fig.add_trace(go.Scatter({}, name="Rust"), row=1, col=1)
    fig.add_trace(go.Scatter({}, name="Cython"), row=1, col=1)
    fig.add_trace(go.Scatter({}, name="Relative"), row=2, col=1)
    fig.update_layout(height=600, width=800, title_text=f"Rust vs Cython - {title}")
    # Log-x/y
    fig.update_xaxes(type="log", title_text=x_label, row=1, col=1)
    fig.update_yaxes(type="log", title_text="time (s)", row=1, col=1)
    
    # Log-x
    fig.update_xaxes(type="log", title_text=x_label, row=2, col=1)
    fig.update_yaxes(title_text="relative time", range=[0, 1.1], row=2, col=1)
    
    return fig

## Fixed k = 10 experiment

In [ ]:
params = fixed_k(10, 15, 40)
fig = setup_plots('Fixed k = 10', 'N')
fig

In [ ]:
for partial_result in run_experiment(params, 3):
   plot_data(partial_result, fig, 'N')

## Proportional k = 0.01% experiment

In [ ]:
params = proportional_k(0.001, 20, 35)
fig = setup_plots('Proportional k = 0.1%', 'N')
fig

In [ ]:
for partial_result in run_experiment(params, 3):
   plot_data(partial_result, fig, 'N')

## Fixed N = 100,000 experiment

In [ ]:
params = fixed_n(100_000, 1, 22)
fig = setup_plots('Fixed N = 100,000', 'k')
fig

In [ ]:
for partial_result in run_experiment(params, 3):
   plot_data(partial_result, fig, 'k')